In [ ]:
!pip install ftfy regex tqdm torch
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch-lightning

In [29]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
import open_clip


In [30]:

open_clip.list_pretrained()


[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion400m_avg'),
 ('ViT-B-32-quickgelu', 'openai'),
 ('ViT-B-32-quickgelu', 'laion400m_e31'),
 ('ViT-B-32-quickgelu', 'laion400m_e32'),
 ('ViT-B-32-quickgelu', 'laion400m_avg'),
 ('ViT-B-16', 'openai'),
 ('ViT-B-16', 'laion400m_e31'),
 ('ViT-B-16', 'laion400m_e32'),
 ('ViT-B-16-plus-240', 'laion400m_e31'),
 ('ViT-B-16-plus-240', 'laion400m_e32'),
 ('ViT-L-14', 'openai'),
 ('ViT-L-14-336', 'openai')]

In [ ]:
#size / architecture of clip model 
clip_model_size = "ViT-B-32"


#model checkpoint
checkpoint= "openai"

#Edit this to specify the augmented/original captions you want to test the model checkpoints on 
caption_to_test =  ["caption"]


#path to validation data after preporcessing into csv file format 
dataframe_path = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv"

In [ ]:
model,_,preprocess = open_clip.create_model_and_transforms(clip_model_size,pretrained=checkpoint)


In [ ]:
clip_model = ALIPRODUCT_CLIP(model)

In [ ]:
def test_feature(caption_col):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,CONFIG.test_batch_size,preprocess,CONFIG.global_random_state,test=True)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    faiss_index = faiss.IndexFlatIP(image_embed.shape[1])
    faiss_index_gpu = faiss.index_cpu_to_all_gpus(faiss_index)
    faiss_index_gpu.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred",sum(top10_preds))

    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [ ]:
# def topk_accuracy(y_true, pred, label_map, k,threshold):
#     pred[pred < threshold] = 0
#     pred_count = []
#     remap = np.vectorize(lambda x: label_map[x]["label"])
#     #get top k values
#     top_k_pred = remap(pred.argsort(axis=1)[:, -k:][:, ::-1])
#     # append 1 if true label is in top k values else append 0
#     for i in range(len(y_true)):
#         if y_true[i] in top_k_pred[i]:
#             pred_count.append(1)
#         else:
#             pred_count.append(0)
#     return sum(pred_count)/len(pred_count)

In [ ]:
col_name = []
top5_acc_preds = []
top10_acc_preds = []
for col in caption_to_test:
    top5_acc,top10_acc = test_feature(col)
    col_name.append(col)
    top5_acc_preds.append(top5_acc)
    top10_acc_preds.append(top10_acc)
pred_df = {"caption_clean_method":col_name,"top_5":top5_acc_preds,"top_10":top10_acc_preds}

In [ ]:
pred_df = pd.DataFrame(pred_df)

In [ ]:
# clip_model = ALIPRODUCT_CLIP(model)
# checkpoint = torch.load("../model_weights/ViT_large.pt")
# for key in list(checkpoint["state_dict"].keys()):
#     checkpoint["state_dict"][key.replace('module.', 'model.')] = checkpoint["state_dict"].pop(key)
# clip_model.load_state_dict(checkpoint["state_dict"])

In [ ]:
# top5_acc_trained_preds = []
# top10_acc_trained_preds = []
# for col in col_to_test:
#     top5_acc,top10_acc = test_feature(col)
#     top5_acc_trained_preds.append(top5_acc)
#     top10_acc_trained_preds.append(top10_acc)
# pred_df["top_5_trained"] = top5_acc_trained_preds
# pred_df["top_10_trained"] = top10_acc_trained_preds

In [ ]:
pred_df